In [ ]:
!pip install /kaggle/input/timm-pytorch-image-models/pytorch-image-models-master

In [ ]:
!pip install -U transformers --no-index --find-links=file:///kaggle/input/huggingfaces/transformers==4.8.1

In [ ]:
import transformers
transformers.__version__

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
sys.path.append("/kaggle/input/d/kurupical/commonlit-kurupical-src/exp")
from transformers import AutoTokenizer
import torch
import tqdm
import gc
import pickle
from exp007 import Config, CommonLitModule, CommonLitDataset
from exp054 import Config, CommonLitModule, CommonLitDataset, TemporalConvNet
import glob

In [ ]:
debug = False

In [ ]:
def feature_engineering(df):
    def total_words(x):
        return len(x.split(" "))

    def total_unique_words(x):
        return len(np.unique(x.split(" ")))

    def total_charactors(x):
        x = x.replace(" ", "")
        return len(x)

    def total_sentence(x):
        x = x.replace("!", "[end]").replace("?", "[end]").replace(".", "[end]")
        return len(x.split("[end]"))

    df_ret = df[["id", "excerpt", "target", "standard_error"]]
    excerpt = df["excerpt"].values
    df_ret["total_words"] = [total_words(x) for x in excerpt]
    df_ret["total_unique_words"] = [total_unique_words(x) for x in excerpt]
    df_ret["total_characters"] = [total_charactors(x) for x in excerpt]
    df_ret["total_sentence"] = [total_sentence(x) for x in excerpt]

    df_ret["div_sentence_characters"] = df_ret["total_sentence"] / df_ret["total_characters"]
    df_ret["div_sentence_words"] = df_ret["total_sentence"] / df_ret["total_words"]
    df_ret["div_characters_words"] = df_ret["total_characters"] / df_ret["total_words"]
    df_ret["div_words_unique_words"] = df_ret["total_words"] / df_ret["total_unique_words"]

    for i, word in enumerate(["!", "?", "(", ")", "'", '"', ";", ".", ","]):
        df_ret[f"count_word_special_{i}"] = [x.count(word) for x in excerpt]

    return df_ret.fillna(0)

In [ ]:
if debug:
    # fold: 0
    df_test = pd.read_csv("/kaggle/input/commonlit-train-fold/train_folds.csv")
    df_test = df_test[df_test["kfold"] == 0]
    df_test = feature_engineering(df_test)
else:
    df_test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")   
    df_test["target"] = 0
    df_test["standard_error"] = 1    
    df_test = feature_engineering(df_test)


In [ ]:
def get_model(exp_name, fold):
    model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit/{exp_name}/best_fold{fold}.ckpt"
    if exp_name == "exp030_roberta_base_bilstm":
        from exp030 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", rnn_module_shrink_ratio=0.25)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    if exp_name == "exp039_roberta_base":
        from exp039 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    if exp_name == "exp040_bert_base_cased":
        from exp040 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    if exp_name == "exp040_roberta_base_without_finetune":
        from exp040 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    if exp_name == "exp046_roberta_base_mlm":
        from exp046 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    if exp_name == "exp045_bert_base_uncased":
        from exp045 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-uncased"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp045_deberta_base":
        from exp045 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/deberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp045_deberta_base":
        from exp045 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/deberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp054_roberta_large_mlm":
        model_path = f"/kaggle/input/exp054-roberta-large-mlm/best_fold{fold}.ckpt"
        from exp054 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", tcn_module_num=2, tcn_module_kernel_size=3, tcn_module_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp053_roberta_base_tcn_residual":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp053 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", tcn_module_enable=True, tcn_module_num=3, tcn_module_kernel_size=4)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp060_roberta_base_attn":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp060 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", tcn_module_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp062_roberta_large_mlm":
        model_path = f"/kaggle/input/exp062-roberta-large-mlm/best_fold{fold}.ckpt"
        from exp062 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", tcn_module_num=3, tcn_module_kernel_size=4, tcn_module_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp056_roberta_base_dense":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp056 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", linear_vocab_dim=8)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp066_roberta_base_without_finetune_bilstm":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp066 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", rnn_module_num=1, 
                     prep_enable=False, hidden_stack_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp066_roberta_large_without_finetune":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp066 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", prep_enable=False, 
                     hidden_stack_enable=True, tcn_module_enable=True, linear_vocab_dim=2, linear_vocab_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp082_roberta_large_bilstm":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp082 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", rnn_module_num=1, 
                     prep_enable=True, hidden_stack_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp079_bert_base_uncased":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp079 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-uncased"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", rnn_module_num=1, 
                     prep_enable=True, hidden_stack_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp077_roberta_base_bilstm":
        model_path = f"/kaggle/input/d/kurupical/kurupical-commonlit2/{exp_name}/best_fold{fold}.ckpt"
        from exp077 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", rnn_module_num=1, 
                     prep_enable=True, hidden_stack_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    
    if exp_name == "exp043":
        model_path = f"/kaggle/input/kurupical-commonlit3/{exp_name}/best_fold{fold}.ckpt"
        from exp043 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer)
        
        model.load_state_dict(torch.load(model_path))
        return model, dataset
    
    
    if exp_name == "exp047_roberta_base":
        model_path = f"/kaggle/input/exp047-roberta-base/best_fold{fold}.ckpt"
        from exp047 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path))
        return model, dataset
    
    
    if exp_name == "exp060":
        model_path = f"/kaggle/input/kurupical-commonlit3/{exp_name}/best_fold{fold}.ckpt"
        from exp060 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="")
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    
    if exp_name == "exp062_roberta_large":
        model_path = f"/kaggle/input/exp062-roberta-large/best_fold{fold}.ckpt"
        from exp062 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", linear_vocab_dim=16, linear_vocab_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path))
        return model, dataset
    
    if exp_name == "exp063":
        model_path = f"/kaggle/input/kurupical-commonlit3/{exp_name}/best_fold{fold}.ckpt"
        from exp063 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", tcn_module_enable=True)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp084_bert_base_cased":
        model_path = f"/kaggle/input/kurupical-commonlit3/{exp_name}/best_fold{fold}.ckpt"
        from exp084 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        cfg = Config(nlp_model_name=nlp_model_path, experiment_name="", prep_enable=True, hidden_stack_enable=True, rnn_module_num=1)
        cfg.fine_tuned_path = None
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp091_roberta_large":
        from exp091 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp091-roberta-large/{exp_name}"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp110_roberta_large":
        from exp110 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp110-roberta-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp116_roberta_large":
        from exp116 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp116-roberta-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp119_deberta_large":
        from exp119 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp119-deberta-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/deberta/large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

        
    if exp_name == "exp126_luke_base":
        from exp126 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp126-luke-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    
    if exp_name == "exp125_deberta_large":
        from exp125 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp125-deberta-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/deberta/large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp134_roberta1" or exp_name == "exp134_roberta2" or exp_name == "exp134_roberta3":
        from exp134 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/seed-variance-test/{exp_name}"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp129_luke_large":
        from exp129 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp129-luke-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp124_bert_base_cased":
        from exp129 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp124-bert-base-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp138_roberta_large_another_fold":
        from exp133 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp138-roberta-large-another-fold"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp139_roberta_base":
        from exp139 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp139-roberta-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    if exp_name == "exp131_luke_base":
        from exp131 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp131-luke-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp139_roberta_base2":
        from exp139 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp139-roberta-base2"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp141_roberta_base":
        from exp141 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp141-roberta-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp143_roberta_base_lstm":
        from exp141 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp143-roberta-base-lstm"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, 
    

    if exp_name == "exp149_roberta_base":
        from exp141 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp149-roberta-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp144_roberta_base_vocab":
        from exp144 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp144-roberta-base-vocab"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp144_2_roberta_base_vocab":
        from exp144 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp144-2-roberta-base-vocab"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp142_2_roberta_base_stack":
        from exp142 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp142-2-roberta-base-stack"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp155_roberta_base_feature":
        from exp155 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp155-roberta-base-feature"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp156_bert_base_cased":
        from exp129 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp156-bert-base-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp156_luke_base":
        from exp156 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp156-luke-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp162_roberta_large":
        from exp162 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp162-roberta-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

        
    if exp_name == "exp162_roberta_large2":
        from exp162 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp162-roberta-large2"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp163_bert_base_uncased":
        from exp129 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp163-bert-base-uncased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-uncased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp165_xlnet_base_cased":
        from exp165 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp165-xlnet-base-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/xlnetbasecased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp164_bert_large_cased":
        from exp129 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp164-bert-large-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-large-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp165_2_xlnet_base_cased":
        from exp165 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp165-2-xlnet-base-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/xlnetbasecased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp164_luke_large":
        from exp164 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp164-luke-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp167_bert_large_uncased":
        from exp167 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp167-bert-large-uncased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-large-uncased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp167_xlnet_large_cased_2":
        from exp167 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp167-xlnet-large-cased-2"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/xlnet-large-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp171_roberta_base":
        from exp171 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp171-roberta-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp170_luke_large":
        from exp170 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp170-luke-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    if exp_name == "exp171_luke_base":
        from exp171 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp171-luke-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp170_bert_large_cased":
        from exp170 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp170-bert-large-cased-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-large-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp176_roberta_base":
        from exp176 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp176-roberta-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
        
    if exp_name == "exp177_roberta_large":
        from exp177 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp177-roberta-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
        
    if exp_name == "exp501_roberta_large":
        from exp501 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp501-roberta-large"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp501_luke_large":
        from exp501 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp501-luke-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp502_bert_base_cased":
        from exp502 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp502-bert-base-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp179_xlnet_base_cased":
        from exp179 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp179-xlnet-base-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/xlnetbasecased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp180_roberta_large":
        from exp180 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp180-roberta-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp180_luke_large":
        from exp180 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp180-luke-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp181_bert_base_cased":
        from exp181 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp181-bert-base-cased"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp181_bert_base_cased2":
        from exp181 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp181-bert-base-cased2"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp181_deberta_base":
        from exp181 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp181-deberta-base"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/deberta-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp180_deberta_large":
        from exp180 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp180-deberta-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/deberta/large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp182_deberta_large":
        from exp182 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp182-deberta-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/deberta/large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp182_roberta_large":
        from exp182 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp182-roberta-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp183_roberta_large":
        from exp183 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp183-roberta-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp180_bert_large_cased":
        from exp180 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp180-bert-large-cased-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-large-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp185_funnel_large":
        from exp185 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp185-funnel-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-large-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp185_funnel_large2":
        from exp185 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp185-funnel-large2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-large-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp192_electra_large":
        from exp192 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp192-electra-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/electra-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp192_electra_large2":
        from exp192 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp192-electra-large2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/electra-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp197_funnel_medium":
        from exp197 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp197-funnel-medium-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-medium-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp191_funnel_large":
        from exp191 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp191-funnel-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp207_electra_large":
        from exp207 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp207-electra-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/electra-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp174_luke_base":
        from exp174 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp174-luke-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp202_deberta_large":
        from exp202 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp202-deberta-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/deberta/large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp211_bert_large_cased2":
        from exp211 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp211-bert-large-cased2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-large-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp214_luke_base":
        from exp214 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp214-luke-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp217_gpt2_medium":
        from exp217 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp217-gpt2-medium-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp216_gpt2_medium":
        from exp216 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp216-gpt2-medium-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp217_gpt2_medium2":
        from exp217 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp217-gpt2-medium2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp227_electra_large":
        from exp227 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp227-electra-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/electra-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp223_bert_base_uncased":
        from exp223 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp223-bert-base-uncased-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/bert-base-uncased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp228_gpt2_medium":
        from exp228 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp228-gpt2-medium-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp228_gpt2_medium2":
        from exp228 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp228-gpt2-medium2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp235_xlnet_large_cased":
        from exp235 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp235-xlnet-large-cased-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/xlnet-large-cased"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp238_funnel_large":
        from exp238 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp238-funnel-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp238_funnel_large2":
        from exp238_2 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp238-funnel-large2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp240_funnel_large_base":
        from exp240 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp240-funnel-large-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-large-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp242_electra_large":
        from exp242 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp242-electra-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/electra-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp242_electra_large2":
        from exp242 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp242-electra-large2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/electra-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp238_funnel_large3":
        from exp238_2 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp238-funnel-large3-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp201_electra_base":
        from exp201 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp201-electra-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/electra-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp250_gpt2_medium":
        from exp250 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp250-gpt2-medium-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp250_gpt2_medium2":
        from exp250 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp250-gpt2-medium2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp252_gpt2_medium":
        from exp252 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp252-gpt2-medium-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp252_gpt2_medium2":
        from exp252 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp252-gpt2-medium2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-medium"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp251_luke_large":
        from exp251 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp251-luke-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp251_luke_large2":
        from exp251 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp251-luke-large2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp251_luke_large3":
        from exp251 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp251-luke-large3-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp251_luke_large4":
        from exp251 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp251-luke-large4-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/luke-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp256_ernie_large":
        from exp251 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp256-ernie-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/ernie-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    
    if exp_name == "exp253_deberta_large":
        from exp253 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp253-deberta-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/deberta/large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp257_funnel_xlarge_base":
        from exp257 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp257-funnel-xlarge-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-xlarge-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp268_mpnet_base":
        from exp268 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp268-mpnet-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/mpnet-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    if exp_name == "exp264_roberta_base":
        from exp264 import Config, CommonLitModule, CommonLitDataset
        nlp_model_path = "/kaggle/input/roberta-transformers-pytorch/roberta-base"
        model_dir = f"/kaggle/input/exp264-roberta-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp273_funnel_xlarge_base":
        from exp273 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp273-funnel-xlarge-base-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/funnel-xlarge-base"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset
    
    if exp_name == "exp275_gpt2_large":
        from exp275 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp275-gpt2-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp278_gpt2_large":
        from exp278 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp278-gpt2-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp278_gpt2_large2":
        from exp278 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp278-gpt2-large2-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    
    if exp_name == "exp288_ernie_large":
        from exp288 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp288-ernie-large-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/ernie-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset


    if exp_name == "exp278_gpt2_large3":
        from exp278 import Config, CommonLitModule, CommonLitDataset
        model_dir = f"/kaggle/input/exp278-gpt2-large3-nb"
        model_path = f"{model_dir}/best_fold{fold}.ckpt"
        cfg_path = f"{model_dir}/cfg.pickle"
        nlp_model_path = "/kaggle/input/huggingface-pretrained-files/gpt2-large"
        with open(cfg_path, "rb") as f:
            cfg = pickle.load(f)
        cfg.nlp_model_name = nlp_model_path
        model = CommonLitModule(cfg=cfg, output_dir="").to("cuda")
        dataset = CommonLitDataset(df=df_test, tokenizer=model.tokenizer, cfg=cfg)
        
        model.load_state_dict(torch.load(model_path)["state_dict"])
        return model, dataset

    

In [ ]:
def get_prediction(exp_name, n_multidropout=1, folds=[0, 1, 2, 3, 4]):
    df = df_test[["id"]]
    
    if debug:
        folds = [0]
    
    print(f"--------- {exp_name} ---------")
    for fold in folds:
        with torch.no_grad():
            print(f"--------- fold: {fold} ---------")
            model, dataset = get_model(exp_name, fold=fold)
            if "deberta_large" in exp_name or "gpt2_medium" in exp_name:
                dataloader = torch.utils.data.DataLoader(dataset, batch_size=4)
            elif "gpt2_large" in exp_name:
                dataloader = torch.utils.data.DataLoader(dataset, batch_size=2)
            else:
                dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

            if n_multidropout == 1:
                model.eval()
            for i in range(n_multidropout):
                pred = []
                for batch in dataloader:
                    if "mlm" in exp_name or "exp060" in exp_name or "exp053" in exp_name or "exp056" in exp_name or "exp066" in exp_name or "exp077" in exp_name or "exp079" in exp_name or "exp082" in exp_name or "exp062" in exp_name or "exp063" in exp_name or "exp047" in exp_name or "exp084" in exp_name or "exp091" in exp_name:
                        input_ids_masked = batch[0].to("cuda")
                        attention_mask = batch[1].to("cuda")
                        token_type_ids = batch[2].to("cuda")
                        input_ids = batch[3].to("cuda")
                        output = model(input_ids_masked, attention_mask, token_type_ids, input_ids)
                    elif "exp045" in exp_name:
                        input_ids = batch[0].to("cuda")
                        attention_mask = batch[1].to("cuda")
                        output = model(input_ids, attention_mask)
                    elif  "exp110" in exp_name or "exp116" in exp_name or "exp119" in exp_name or "exp125" in exp_name or "exp126" in exp_name or "exp134" in exp_name or "exp129" in exp_name or "exp124" in exp_name or "exp138" in exp_name or "exp139" in exp_name or "exp131" in exp_name or "exp141" in exp_name:
                        input_ids_masked = batch[0].to("cuda")
                        attention_mask = batch[1].to("cuda")
                        token_type_ids = batch[2].to("cuda")
                        input_ids = batch[3].to("cuda")
                        output, _ = model(input_ids_masked, attention_mask, token_type_ids, input_ids)                    
                    elif "exp288" in exp_name or "exp278" in exp_name or "exp275" in exp_name or "exp273" in exp_name or "exp264" in exp_name or "exp268" in exp_name or "exp257" in exp_name or "exp253" in exp_name or "exp256" in exp_name or "exp251" in exp_name or "exp252" in exp_name or "exp250" in exp_name or "exp201" in exp_name or "exp242" in exp_name or "exp240" in exp_name or "exp238" in exp_name or "exp235" in exp_name or "exp228" in exp_name or "exp223" in exp_name or "exp227" in exp_name or "exp216" in exp_name or "exp217" in exp_name or "exp214" in exp_name or "exp211" in exp_name or "exp202" in exp_name or "exp174" in exp_name or "exp207" in exp_name or "exp191" in exp_name or "exp197" in exp_name or "exp192" in exp_name or "exp185" in exp_name or "exp183" in exp_name or "exp182" in exp_name or "exp181" in exp_name or "exp179" in exp_name or "exp180" in exp_name or "exp155" in exp_name or "exp156_luke_base" in exp_name or "exp162" in exp_name or "exp165" in exp_name or "exp164_luke_large" in exp_name or "exp167" in exp_name or "exp171" in exp_name or "exp170" in exp_name or "exp174" in exp_name or "exp176" in exp_name or "exp177" in exp_name or "exp501" in exp_name or "exp502" in exp_name:
                        input_ids_masked = batch[0].to("cuda")
                        attention_mask = batch[1].to("cuda")
                        token_type_ids = batch[2].to("cuda")
                        input_ids = batch[3].to("cuda")
                        features = batch[4].to("cuda")
                        output, _ = model(input_ids_masked, attention_mask, token_type_ids, input_ids, features)                                        
                    elif "exp040" in exp_name:                    
                        input_ids = batch[0].to("cuda")
                        attention_mask = batch[1].to("cuda")
                        token_type_ids = batch[2].to("cuda")
                        output = model(input_ids, attention_mask, token_type_ids)
                    else:
                        input_ids_masked = batch[0].to("cuda")
                        attention_mask = batch[1].to("cuda")
                        token_type_ids = batch[2].to("cuda")
                        input_ids = batch[3].to("cuda")
                        output, _ = model(input_ids_masked, attention_mask, token_type_ids, input_ids)                                        
                    pred.extend(output.detach().cpu().numpy().flatten().tolist())
                    del output
                    gc.collect()
                    torch.cuda.empty_cache()
                df[f"pred_fold{fold}_i{i}"] = pred
            del model, dataset, dataloader
            gc.collect()
            torch.cuda.empty_cache()
    print(df)
    df[exp_name] = df.drop("id", axis=1).mean(axis=1)
    
    return df[exp_name].values

In [ ]:
def make_predict(model_dict):
    pred = np.zeros(len(df_test))
    
    for model, weight in model_dict.items():
        if model == "coef":
            pred += weight
        else:
            pred += get_prediction(model) * weight
    return pred

In [ ]:
pred1 = make_predict(model_dict={
 'exp192_electra_large2': 0.08295192073911264,
 'exp191_funnel_large': 0.12150247993915793,
 'exp202_deberta_large': 0.15167895227258077,
 'exp228_gpt2_medium': 0.12193089040813881,
 'exp251_luke_large': 0.09370238709826298,
 'exp253_deberta_large': 0.11799288574092036,
 'exp268_mpnet_base': 0.09995350620369359,
 'exp288_ernie_large': 0.07922850022202096,
 'exp278_gpt2_large3': 0.16487686471775878,
 'coef': 0.009468716183650404
})
pred = pred1
# pred = pred1*0.9 + pred2*0.1

In [ ]:
df = df_test[["id"]]
df["target"] = pred

In [ ]:
if debug:
    target = df_test["target"].values
    rmse = np.sqrt(1 / len(pred) * ((target - pred)**2).sum())
    print(rmse)

In [ ]:
df[["id", "target"]].to_csv("submission.csv", index=False)

In [ ]:
df.head(10)